# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [2]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [3]:
# Делаю pulled OLS ради интереса
pulled_ols = feols(log(v) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df)
pulled_ols %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           8.748607   0.301364 29.030005  < 2.2e-16
log(deaths_i)                         0.325192   0.090876  3.578422 0.00044799
log(deaths_j)                         0.150638   0.065264  2.308131 0.02216840
C8EV_International_travel_controls_i  0.234891   0.157781  1.488711 0.13837363
C8EV_International_travel_controls_j -0.020201   0.137475 -0.146944 0.88334648
                                        
(Intercept)                          ***
log(deaths_i)                        ***
log(deaths_j)                        *  
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.09796   Adj. R2: 0.034332

In [4]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(v) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
log(deaths_i)                        -0.013392   0.009686 -1.382547  0.16858 
log(deaths_j)                        -0.000571   0.008342 -0.068415  0.94553 
C8EV_International_travel_controls_i -0.023257   0.020419 -1.138968  0.25628 
C8EV_International_travel_controls_j -0.002680   0.017059 -0.157115  0.87534 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.53245     Adj. R2: 0.628924
                Within R2: 2.681e-5

In [5]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(v) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                       0.022962   0.115784  0.198314
year2022                                       0.070461   0.124519  0.565864
year2020:log(deaths_i)                        -0.011340   0.011314 -1.002314
year2021:log(deaths_i)                        -0.016472   0.010987 -1.499205
year2022:log(deaths_i)                        -0.019750   0.011870 -1.663835
year2020:log(deaths_j)                        -0.008217   0.010192 -0.806233
year2021:log(deaths_j)                         0.002133   0.010104  0.211126
year2022:log(deaths_j)                         0.010205   0.011284  0.904311
year2020:C8EV_International_travel_controls_i -0.071633   0.034438 -2.080066
year2021:C8EV_International_travel_controls_i -0.023306   0.020801 -1.120400
year2022:C8EV_International_tra

In [6]:
# С кластеризацией ошибок есть значимый эффект и наблюдается гетерогенность во времени (+ направление связи соотносится с описанным в литературе)
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i+j, signif.code=signif.code)
# А чем такая кластеризация отличается от i^j?

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,8.749*** (0.3014),,
4,log(deaths_i),0.3252*** (0.0909),-0.0134 (0.0097),
5,log(deaths_j),0.1506* (0.0653),-0.0006 (0.0083),
6,C8EV_International_travel_controls_i,0.2349 (0.1578),-0.0233 (0.0204),
7,C8EV_International_travel_controls_j,-0.0202 (0.1375),-0.0027 (0.0171),
8,year2021,,,0.0230 (0.1158)
9,year2022,,,0.0705 (0.1245)


In [7]:
# При использовании НАС ошибок все эффекты становятся незначимыми
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code=signif.code)

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,8.749*** (0.0565),,
4,log(deaths_i),0.3252** (0.0129),-0.0134 (0.0136),
5,log(deaths_j),0.1506** (0.0118),-0.0006 (0.0140),
6,C8EV_International_travel_controls_i,0.2349* (0.0298),-0.0233 (0.0279),
7,C8EV_International_travel_controls_j,-0.0202 (0.0303),-0.0027 (0.0274),
8,year2021,,,0.0230 (0.1856)
9,year2022,,,0.0705 (0.1981)


In [8]:
# Всё-таки кластеризовать только по году нельзя, потому что мы в формуле указываем кусочки ошибок которые скоррелированные
    # Пробую кластеризацию ошибок по годам: с теоретической точки зрения это обосновано тем,
    # что все связанные с одной страной наблюдения связаны, значит и во всём графе наблюдения связаны.
    # При этом если нет автокорреляции (учитывая, что у меня 3 периода, это вроде ОК), то кластеризация по годам именно так и делает.
    # Но в статьях пишут, что нельзя использовать кластеризацию ошибок, если меньше чем 50 кластеров...
    # Интересно, что здесь эффект импортёра значимый, а в предыдущей спецификации эффект экспортёра...
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i+j+year, signif.code=signif.code)

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).



,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,8.749* (0.9627),,
4,log(deaths_i),0.3252. (0.1003),-0.0134*** (1e-5),
5,log(deaths_j),0.1506 (0.0827),-0.0006*** (1e-5),
6,C8EV_International_travel_controls_i,0.2349 (0.1954),-0.0233*** (1e-5),
7,C8EV_International_travel_controls_j,-0.0202 (0.1655),-0.0027*** (1e-5),
8,year2021,,,0.0230 (0.0086)
9,year2022,,,0.0705* (0.0120)


In [9]:
# Не понятно, насколько это осмысленно, но сделал на всякий случай
etable(pulled_ols, fe_no_iv, heterogenous_fe, vcov = ~i^year+j^year, signif.code=signif.code)

,,pulled_ols,fe_no_iv,heterogenous_fe
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,8.749*** (0.3347),,
4,log(deaths_i),0.3252*** (0.0634),-0.0134 (0.0155),
5,log(deaths_j),0.1506** (0.0461),-0.0006 (0.0180),
6,C8EV_International_travel_controls_i,0.2349. (0.1421),-0.0233 (0.0313),
7,C8EV_International_travel_controls_j,-0.0202 (0.1340),-0.0027 (0.0289),
8,year2021,,,0.0230 (0.1639)
9,year2022,,,0.0705 (0.1865)


### Инструментальные переменные на панельных данных

In [10]:
# Pulled TSLS
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
pulled_tsls = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) 
 ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
population_density + population_density_j
  ,
 data = df
 )
pulled_tsls %>% summary(vcov = ~i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           7.558642   0.555417 13.608954 < 2.2e-16
fit_log(deaths_i)                    -0.187273   0.569224 -0.328997   0.74255
fit_log(deaths_j)                    -0.373980   0.617327 -0.605806   0.54543
C8EV_International_travel_controls_i  0.319239   0.239136  1.334971   0.18363
C8EV_International_travel_controls_j  0.037631   0.227472  0.165429   0.86880
                                        
(Intercept)                          ***
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_travel_

In [11]:
# Только FE года (инструменты эндогенные)
fe_iv_year = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year|
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j 
+
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
  population_density + population_density_j
 ,
 data = df
 )
fe_iv_year %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                     Estimate Std. Error t value   Pr(>|t|)    
fit_log(deaths_i)                    1.016266   0.148505 6.84330 1.2697e-10 ***
fit_log(deaths_j)                    0.624096   0.115088 5.42276 1.9388e-07 ***
C8EV_International_travel_controls_i 1.204183   0.270775 4.44718 1.5470e-05 ***
C8EV_International_travel_controls_j 0.833436   0.201166 4.14302 5.3404e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.27872     Adj. R2: -0.051569
                Within R2: -0.051726
F-test (1st stage), log(deaths_i): stat = 7,515.0, p < 2.2e-16, on 6 and 56,392 DoF.
F-test (1st stage), lo

In [13]:
feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.013846   0.031045 -0.446018  0.65614 
fit_log(deaths_j)                    -0.000788   0.029761 -0.026472  0.97891 
C8EV_International_travel_controls_i -0.021464   0.024483 -0.876701  0.38186 
C8EV_International_travel_controls_j  0.001011   0.019822  0.051024  0.95936 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.53447     Adj. R2: 0.628745
                Within R2: 2.504e-5
F-test (1st stage), log(deaths_i): stat = 738.3     , p

In [14]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 
# Добавить больше инструментов, которые меняются во времени. Мб плотность населения или медианный возраст брать не за 2019 год а в динамике, 
# чтобы использовать их как инструменты в этой спецификации.
fe_iv = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.014023   0.032718 -0.428591  0.66875 
fit_log(deaths_j)                     0.009889   0.030230  0.327118  0.74397 
C8EV_International_travel_controls_i -0.021767   0.024814 -0.877200  0.38159 
C8EV_International_travel_controls_j  0.005725   0.018770  0.305018  0.76072 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.53448     Adj. R2: 0.628741
                Within R2: 1.4e-5  
F-test (1st stage), log(deaths_i): stat = 948.9    , p < 2.2e-16 , on 6 and 56,044 DoF.
F-test (1st sta

In [15]:
fe_iv_age = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
median_age_i + median_age_j
 ,
 data = df
 )
fe_iv_age %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/47).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j
Second stage: Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                     Estimate Std. Error  t value Pr(>|t|)    
fit_log(deaths_i)                    0.062870   0.037782 1.664004 0.097912 .  
fit_log(deaths_j)                    0.013908   0.024925 0.557997 0.577563    
C8EV_International_travel_controls_i 0.007723   0.024735 0.312216 0.755250    
C8EV_International_travel_controls_j 0.002525   0.018784 0.134415 0.893230    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.53325     Adj. R2:  0.628689
                Within R2: -6.044e-4
F-test (1st stage), log(deaths_i): stat =   946.4     , p < 2.2e-16, on 2 and 56,386 DoF.
F-test (1st stage), log(deaths_j): stat = 1,028.4     , p < 2.2e-16, on 2 and 56,386 DoF.
    

In [16]:
fe_iv_age_temp = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
median_age_i + median_age_j
+
average_surface_temperature_i + average_surface_temperature_j 
 ,
 data = df
 )
fe_iv_age_temp %>% summary(cluster = ~ i+j)

NOTE: 17,245 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/8,783).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, average_surface_temperature_i, average_surface_temperature_j
Second stage: Dep. Var.: log(v)
Observations: 53,062
Fixed-effects: year: 3,  i: 166,  j: 166
Standard-errors: Clustered (i & j) 
                                     Estimate Std. Error  t value Pr(>|t|) 
fit_log(deaths_i)                    0.050013   0.036093 1.385651  0.16772 
fit_log(deaths_j)                    0.019970   0.025539 0.781922  0.43538 
C8EV_International_travel_controls_i 0.005865   0.023861 0.245780  0.80616 
C8EV_International_travel_controls_j 0.008908   0.019118 0.465933  0.64188 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.5157     Adj. R2:  0.628299
               Within R2: -4.844e-4
F-test (1st stage), log(deaths_i): stat = 472.0     , p < 2.2e-16 , on 4 and 52,725 DoF.
F-test (1st stage), log(deaths_j): stat = 501.1   

In [17]:
iv_both_endo = feols(log(v) ~  1 | year+i+j|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_both_endo %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 13,568/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                         0.048098   0.037600  1.279196
fit_log(deaths_j)                         0.005798   0.034050  0.170285
fit_C8EV_International_travel_controls_i -0.165073   0.103613 -1.593180
fit_C8EV_International_travel_controls_j  0.003470   0.077292  0.044895
                                         Pr(>|t|) 
fit_log(deaths_i)                         0.20253 
fit_log(deaths_j)                         0.86498 
fit_C8EV_International_travel_controls_i  0

In [18]:
# Даже тест Вальда говорит, что ничего не значимо...
wald(iv_both_endo, keep = 'log|C8', vcov = ~i+j)

Wald test, H0: joint nullity of fit_log(deaths_i), fit_log(deaths_j), fit_C8EV_International_travel_controls_i and fit_C8EV_International_travel_controls_j
 stat = 0.788987, p-value = 0.53208, on 4 and 56,046 DoF, VCOV: Clustered (i & j).

In [19]:
# Тут всё плохо в тесте Саргана
# Мы верим в синюю схему, поэтому такая спецификация бессмысленна. Утверждение в ВКР: если схема верна, то влияение отрицательное.
feols(log(v) ~  1 | year|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 13,568/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                         Estimate Std. Error  t value
fit_log(deaths_i)                        1.017800   0.165012 6.168040
fit_log(deaths_j)                        0.626120   0.130734 4.789287
fit_C8EV_International_travel_controls_i 0.200526   1.619977 0.123783
fit_C8EV_International_travel_controls_j 0.038021   1.173255 0.032407
                                           Pr(>|t|)    
fit_log(deaths_i)                        4.6996e-09 ***
fit_log(deaths_j)                        3.5657e-06 ***
fit_C8EV_International_travel_controls_i 9.0163e-01    


In [ ]:
iv_both_endo_temperature = feols(log(v) ~  1 | year+i+j|
 log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
+
average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_both_endo_temperature %>% summary(cluster = ~ i+j)
wald(iv_both_endo_temperature, keep = 'log|C8', vcov = ~i+j)

NOTE: 17,571 observations removed because of NA and infinite values (IV: 13,568/15,000).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j), C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j, average_surface_temperature_i, average_surface_temperature_j
Second stage: Dep. Var.: log(v)
Observations: 52,736
Fixed-effects: year: 3,  i: 166,  j: 166
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_log(deaths_i)                        -0.019786   0.064143 -0.308461
fit_log(deaths_j)                         0.045376   0.039469  1.149661
fit_C8EV_International_travel_controls_i -0.032627   0.122657 -0.266001
fit_C8EV_International_travel_controls_j -0.044376   0.082666 -0.536816
                                         Pr(>|t|) 
fit_log(deaths_i)                         0.75812 
fit_log(deaths_j)               

Wald test, H0: joint nullity of fit_log(deaths_i), fit_log(deaths_j), fit_C8EV_International_travel_controls_i and fit_C8EV_International_travel_controls_j
 stat = 0.493658, p-value = 0.740425, on 4 and 52,399 DoF, VCOV: Clustered (i & j).

In [21]:
# Как будто ограничения тоже эндогенные...
# Но в своей схеме я считаю, что они экзогенные, потому что обосновали схему.
# Если бы они были эндогенными, то наша оценка была бы смещена куда-то
iv_itc_endo = feols(log(v) ~  1 + log(deaths_i) + log(deaths_j) | year+i+j|
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
iv_itc_endo %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 6,067, IV: 9,802/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_C8EV_International_travel_controls_i -0.054602   0.073744 -0.740420
fit_C8EV_International_travel_controls_j  0.016396   0.060523  0.270902
log(deaths_i)                            -0.016555   0.012230 -1.353598
log(deaths_j)                             0.001426   0.009702  0.147030
                                         Pr(>|t|) 
fit_C8EV_International_travel_controls_i  0.46004 
fit_C8EV_International_travel_controls_j  0.78679 
log(deaths_i)                             0.17762 
log(deaths_j)         

In [22]:
# Как будто ограничения тоже эндогенные...
m8 = feols(log(v) ~  1 | year+i+j|
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m8 %>% summary(cluster = ~ i+j)

NOTE: 10,661 observations removed because of NA values (IV: 9,802/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : C8EV_International_travel_controls_i, C8EV_International_travel_controls_j
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 59,646
Fixed-effects: year: 3,  i: 178,  j: 178
Standard-errors: Clustered (i & j) 
                                          Estimate Std. Error   t value
fit_C8EV_International_travel_controls_i -0.102452   0.088111 -1.162762
fit_C8EV_International_travel_controls_j -0.040139   0.070984 -0.565457
                                         Pr(>|t|) 
fit_C8EV_International_travel_controls_i  0.24649 
fit_C8EV_International_travel_controls_j  0.57248 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.56168     Adj. R2:  0.622218
                Within R2: -2.484e-4
F-test (1st stage), C8EV_International_travel_controls_i: stat = 719.1     ,

In [23]:
m9 = feols(log(v) ~  1 | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m9 %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                   Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i) -0.015813   0.036559 -0.432530  0.66589 
fit_log(deaths_j)  0.012139   0.032889  0.369087  0.71251 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 2.53451     Adj. R2:  0.628747
                Within R2: -5.738e-6
F-test (1st stage), log(deaths_i): stat = 618.4    , p < 2.2e-16 , on 6 and 56,046 DoF.
F-test (1st stage), log(deaths_j): stat = 573.6    , p < 2.2e-16 , on 6 and 56,046 DoF.
                       Wu-Hausman: stat =   0.03291, p = 0.967625, on 2 and 56,046 DoF.
                           Sargan: stat =   1.9582

In [24]:
# Слово влияние это экспертное мнение на основе методов
# Температура это ложная корреляция, не надо
# Колонка 4 --- у нас нет данных, чтобы оценить эту связь. Недоидентифицированная система (всё на всё влияет)
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i+j, signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v),log(v),log(v),log(v)
2,,,,,,,
3,Constant,7.559*** (0.5554),,,,,
4,log(deaths_i),-0.1873 (0.5692),1.016*** (0.1485),-0.0140 (0.0327),0.0481 (0.0376),-0.0198 (0.0641),-0.0165 (0.0122)
5,log(deaths_j),-0.3740 (0.6173),0.6241*** (0.1151),0.0099 (0.0302),0.0058 (0.0340),0.0454 (0.0395),0.0014 (0.0097)
6,C8EV_International_travel_controls_i,0.3192 (0.2391),1.204*** (0.2708),-0.0218 (0.0248),-0.1651 (0.1036),-0.0326 (0.1227),-0.0546 (0.0737)
7,C8EV_International_travel_controls_j,0.0376 (0.2275),0.8334*** (0.2012),0.0057 (0.0188),0.0035 (0.0773),-0.0444 (0.0827),0.0164 (0.0605)
8,Fixed-Effects:,-----------------,------------------,----------------,----------------,----------------,----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


In [25]:
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v),log(v),log(v),log(v)
2,,,,,,,
3,Constant,7.559*** (0.1142),,,,,
4,log(deaths_i),-0.1873 (0.1072),1.016*** (0.0212),-0.0140 (0.0509),0.0481 (0.0918),-0.0198 (0.0902),-0.0165 (0.0161)
5,log(deaths_j),-0.3740. (0.1061),0.6241** (0.0199),0.0099 (0.0449),0.0058 (0.0745),0.0454 (0.0689),0.0014 (0.0164)
6,C8EV_International_travel_controls_i,0.3192* (0.0447),1.204** (0.0428),-0.0218 (0.0357),-0.1651 (0.1557),-0.0326 (0.1447),-0.0546 (0.0950)
7,C8EV_International_travel_controls_j,0.0376 (0.0448),0.8334** (0.0420),0.0057 (0.0331),0.0035 (0.1390),-0.0444 (0.1207),0.0164 (0.0860)
8,Fixed-Effects:,-----------------,-----------------,----------------,----------------,----------------,----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


In [26]:
    # Всё ещё странно использовать кластеризацию по году... 
    # etable(m3, m4, vcov = function(x) vcovHC(x, method="arellano", type="HC1"), signif.code = signif.code) Не сработало
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i+j+year, signif.code = signif.code)

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).



,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_te..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v),log(v),log(v),log(v)
2,,,,,,,
3,Constant,7.559* (1.204),,,,,
4,log(deaths_i),-0.1873 (0.6614),1.016* (0.1772),-0.0140*** (1e-5),0.0481*** (1e-5),-0.0198*** (1e-5),-0.0165*** (1e-5)
5,log(deaths_j),-0.3740 (0.8467),0.6241* (0.1220),0.0099*** (1e-5),0.0058*** (1e-5),0.0454*** (1e-5),0.0014*** (1e-5)
6,C8EV_International_travel_controls_i,0.3192 (0.3054),1.204* (0.2109),-0.0218*** (1e-5),-0.1651*** (1e-5),-0.0326*** (1e-5),-0.0546*** (1e-5)
7,C8EV_International_travel_controls_j,0.0376 (0.2133),0.8334* (0.1547),0.0057*** (1e-5),0.0035*** (1e-5),-0.0444*** (1e-5),0.0164*** (1e-5)
8,Fixed-Effects:,----------------,----------------,-----------------,-----------------,-----------------,-----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


In [27]:
# Так вроде не очень корректно делать, если мы верим в возможную автокорреляцию
etable(pulled_tsls, fe_iv_year, fe_iv, iv_both_endo, iv_both_endo_temperature, iv_itc_endo, vcov = ~i^year+j^year, signif.code = signif.code)

,,pulled_tsls,fe_iv_year,fe_iv,iv_both_endo,iv_both_endo_t..,iv_itc_endo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v),log(v),log(v),log(v)
2,,,,,,,
3,Constant,7.559*** (0.5707),,,,,
4,log(deaths_i),-0.1873 (0.4280),1.016*** (0.0909),-0.0140 (0.0586),0.0481 (0.0757),-0.0198 (0.0912),-0.0165 (0.0173)
5,log(deaths_j),-0.3740 (0.4746),0.6241*** (0.0691),0.0099 (0.0518),0.0058 (0.0651),0.0454 (0.0818),0.0014 (0.0188)
6,C8EV_International_travel_controls_i,0.3192 (0.2006),1.204*** (0.2129),-0.0218 (0.0460),-0.1651 (0.1523),-0.0326 (0.1922),-0.0546 (0.1339)
7,C8EV_International_travel_controls_j,0.0376 (0.1921),0.8334*** (0.1561),0.0057 (0.0429),0.0035 (0.1375),-0.0444 (0.1751),0.0164 (0.1169)
8,Fixed-Effects:,-----------------,------------------,----------------,----------------,----------------,----------------
9,year,No,Yes,Yes,Yes,Yes,Yes


## Три пространственные регрессии (как в курсовой)

Но в такую спецификацию нельзя добавить фиксированные эффекты, поэтому почти наверное без инструментов будет смещение из-за пропущенных переменных

In [28]:
# Без инструментов.
m_2020 = feols(log(v) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 5,531).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 4,126).

NOTE: 3,911 observations removed because of NA and infinite values (RHS: 3,911).



OLS estimation, Dep. Var.: log(v)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                          5.563712   0.202765 27.4392 < 2.2e-16 ***
log(deaths_i)                        0.372474   0.014981 24.8636 < 2.2e-16 ***
log(deaths_j)                        0.244586   0.014349 17.0458 < 2.2e-16 ***
C8EV_International_travel_controls_i 0.895261   0.056778 15.7678 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.680061   0.057191 11.8910 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 3.9653   Adj. R2: 0.064359

OLS estimation, Dep. Var.: log(v)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error  t value   Pr(>|t|)
(Intercept)                          4.294474   0.128804 33.34124  < 2.2e-16
log(deaths_i)                        0.291123   0.019402 15.00443  < 2.2e-16
log(deaths_j)                        0.111539   0.017035  6.54750 5.9973e-11
C8EV_International_travel_controls_i 1.147440   0.038908 29.49124  < 2.2e-16
C8EV_International_travel_controls_j 0.835839   0.039068 21.39448  < 2.2e-16
                                        
(Intercept)                          ***
log(deaths_i)                        ***
log(deaths_j)                        ***
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.03296   Adj. R2: 0.077936

OLS estimation, Dep. Var.: log(v)
Observations: 18,916
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                          8.909318   0.078977 112.80953  < 2.2e-16
log(deaths_i)                        0.483443   0.015579  31.03087  < 2.2e-16
log(deaths_j)                        0.258449   0.014147  18.26831  < 2.2e-16
C8EV_International_travel_controls_i 0.671115   0.072550   9.25034  < 2.2e-16
C8EV_International_travel_controls_j 0.365265   0.066691   5.47697 4.3819e-08
                                        
(Intercept)                          ***
log(deaths_i)                        ***
log(deaths_j)                        ***
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.07348   Adj. R2: 0.061527

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,5.564*** (0.2028),4.294*** (0.1288),8.909*** (0.0790)
4,log(deaths_i),0.3725*** (0.0150),0.2911*** (0.0194),0.4834*** (0.0156)
5,log(deaths_j),0.2446*** (0.0143),0.1115*** (0.0170),0.2584*** (0.0141)
6,C8EV_International_travel_controls_i,0.8953*** (0.0568),1.147*** (0.0389),0.6711*** (0.0726)
7,C8EV_International_travel_controls_j,0.6801*** (0.0572),0.8358*** (0.0391),0.3653*** (0.0667)
8,____________________________________,__________________,__________________,__________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedas.-rob.


In [29]:
# С инструментами оценки коэффициентов всё равно не похожи на панельные данные (пространственные смещённые, т.к должно быть много unobservables)
# Использую ошибки hc1
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
# Почему экзогенные в панельной спецификации инструменты эндогенны в пространственной?
m_2020 = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)
etable(m_2020, m_2021, m_2022, vcov = ~i+j, signif.code = signif.code)

NOTE: 5,531 observations removed because of NA and infinite values (RHS: 3,419, IV: 3,381/3,419).

NOTE: 4,126 observations removed because of NA and infinite values (RHS: 3,290, IV: 1,557/3,290).

NOTE: 4,249 observations removed because of NA and infinite values (RHS: 3,093, IV: 1,129/3,952).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 18,391
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                          5.969386   0.213890 27.9086 < 2.2e-16 ***
fit_log(deaths_i)                    0.902226   0.022833 39.5137 < 2.2e-16 ***
fit_log(deaths_j)                    0.558973   0.020960 26.6685 < 2.2e-16 ***
C8EV_International_travel_controls_i 1.056775   0.061515 17.1791 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.784651   0.060168 13.0409 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.16015   Adj. R2: -0.029856
F-test (1st stage), log(deaths_i): stat = 2,699.8, p < 2.2e-16, on 6 and 18,382 DoF.
F-te

TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 19,432
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                          4.481759   0.137363 32.6271 < 2.2e-16 ***
fit_log(deaths_i)                    1.039850   0.029786 34.9101 < 2.2e-16 ***
fit_log(deaths_j)                    0.568131   0.026997 21.0446 < 2.2e-16 ***
C8EV_International_travel_controls_i 1.146007   0.041462 27.6397 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.833670   0.041619 20.0308 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.31431   Adj. R2: -0.055203
F-test (1st stage), log(deaths_i): stat = 2,100.5, p < 2.2e-16, on 6 and 19,423 DoF.
F-te

TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 18,578
Standard-errors: Heteroskedasticity-robust 
                                     Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)                          8.938381   0.082064 108.9203 < 2.2e-16 ***
fit_log(deaths_i)                    0.871966   0.020231  43.1007 < 2.2e-16 ***
fit_log(deaths_j)                    0.561645   0.019106  29.3965 < 2.2e-16 ***
C8EV_International_travel_controls_i 1.289039   0.078296  16.4637 < 2.2e-16 ***
C8EV_International_travel_controls_j 0.796659   0.073831  10.7904 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 4.18539   Adj. R2: 0.013015
F-test (1st stage), log(deaths_i): stat = 5,398.6, p < 2.2e-16 , on 6 and 18,569 DoF

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,5.969*** (0.2139),4.482*** (0.1374),8.938*** (0.0821)
4,log(deaths_i),0.9022*** (0.0228),1.040*** (0.0298),0.8720*** (0.0202)
5,log(deaths_j),0.5590*** (0.0210),0.5681*** (0.0270),0.5616*** (0.0191)
6,C8EV_International_travel_controls_i,1.057*** (0.0615),1.146*** (0.0415),1.289*** (0.0783)
7,C8EV_International_travel_controls_j,0.7847*** (0.0602),0.8337*** (0.0416),0.7967*** (0.0738)
8,____________________________________,__________________,__________________,__________________
9,S.E. type,Heteroskedas.-rob.,Heteroskedas.-rob.,Heteroskedas.-rob.


,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,Constant,5.969*** (1.265),4.482*** (0.7216),8.938*** (0.5137)
4,log(deaths_i),0.9022*** (0.1570),1.040*** (0.2066),0.8720*** (0.1270)
5,log(deaths_j),0.5590*** (0.1119),0.5681*** (0.1414),0.5616*** (0.1046)
6,C8EV_International_travel_controls_i,1.057* (0.4228),1.146*** (0.2648),1.289* (0.5608)
7,C8EV_International_travel_controls_j,0.7847** (0.2893),0.8337*** (0.1878),0.7967. (0.4445)
8,____________________________________,__________________,__________________,__________________
9,S.E.: Clustered,by: i & j,by: i & j,by: i & j
